In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from math import isnan
from PIL import Image    
import os
import random
import math
from collections import defaultdict
plt.rcParams["figure.figsize"] = (5, 3) # (w, h)
plt.rcParams["figure.dpi"] = 200

In [2]:
path = './Fashion_MNIST'
classes = os.listdir(path)
num_class = len(classes)
im1 = []
classlabel = []
for i in classes:
    path1 = os.path.join(path,i)
    for j in os.listdir(path1):
        im = Image.open(os.path.join(path1,j))
        im = im.convert('L')
        im1.append(np.asarray(im, dtype = np.uint8))
        classlabel.append(i)

In [39]:
data = np.array(im1).reshape(-1,28*28)

In [40]:
np.std(data, axis=0)

array([0.00000000e+00, 5.58142847e-02, 1.36358901e-01, 2.60333320e-01,
       1.74015086e+00, 1.10866597e+01, 1.44511026e+01, 1.53032625e+01,
       2.79112151e+01, 4.70762144e+01, 6.88187585e+01, 7.19335788e+01,
       6.92606247e+01, 6.59473090e+01, 6.41998782e+01, 6.80630561e+01,
       7.08331567e+01, 7.10744229e+01, 6.54679042e+01, 3.61833957e+01,
       1.82975365e+01, 1.28616584e+01, 9.38901817e+00, 8.96580690e+00,
       5.55008358e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.23257139e-01, 2.22204860e-01, 2.96488400e+00,
       1.17211877e+01, 2.29833752e+01, 3.44480189e+01, 5.05288964e+01,
       7.10743424e+01, 8.61080757e+01, 1.01067419e+02, 1.02227306e+02,
       1.05820381e+02, 1.05038396e+02, 1.04525988e+02, 1.06513473e+02,
       1.01771422e+02, 9.80021038e+01, 1.00373437e+02, 8.32064802e+01,
       6.52477292e+01, 4.60265352e+01, 2.31736515e+01, 1.52668116e+01,
       1.82688248e+01, 9.24974609e+00, 0.00000000e+00, 0.00000000e+00,
      

In [37]:
def norm(dataMat):
    average = np.mean(dataMat, axis=0) #按列求均值
    m, n = np.shape(dataMat)
    meanRemoved = dataMat-average #减去均值
    normData = meanRemoved / np.std(dataMat, axis=0) #标准差归一化
    return normData

In [38]:
data = np.array(im1).reshape(-1,28*28)
data = norm(data)

C:\Users\517super\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [12]:
target = np.array(classlabel).reshape(-1,1)
target = target.astype(int)
df = np.hstack((data,target))

In [6]:
df

array([[-1.07309641, -1.07309641, -1.07309641, ..., -1.07309641,
        -1.07309641,  0.        ],
       [-0.92537253, -0.92537253, -0.92537253, ..., -0.92537253,
        -0.92537253,  0.        ],
       [-0.99267235, -0.99267235, -0.99267235, ..., -0.99267235,
        -0.99267235,  0.        ],
       ...,
       [-0.57599479, -0.57599479, -0.57599479, ..., -0.57599479,
        -0.57599479,  4.        ],
       [-0.88080183, -0.88080183, -0.88080183, ..., -0.88080183,
        -0.88080183,  4.        ],
       [-1.11599926, -1.11599926, -1.11599926, ..., -1.11599926,
        -1.11599926,  4.        ]])

In [13]:
np.random.seed(135) 
np.random.shuffle(df)
train, test = df[:-32,:], df[-32:,:]
Xtrain, Xtest = train[:,:-1], test[:,:-1]
y1, y2 = train[:,-1], test[:,-1]

In [14]:
def one_hot(X):
    t1 = []
    for i in X:
        t = [0 for k in range(num_class)]
        t[int(i)] = 1
        t1.append(t)
    return np.array(t1)

In [15]:
ytrain, ytest = one_hot(y1), one_hot(y2)

In [16]:
class LogisticModal(object):
    def __init__(self,Num_class, Num_epoch=50):
        self.Num_epoch = Num_epoch
        self.Num_class = Num_class
    def phi(self,x):
        x = np.reshape(x,(len(x),1))
        return x
    def softmax(self,n,k,X):
        s = np.float64(0.)
        ak = self.w[k,:].T.dot(self.phi(X[n,:]))
        for j in range(self.Num_class):
            aj = self.w[j,:].T.dot(self.phi(X[n,:]))
            s += np.nan_to_num(np.exp(aj - ak))
        s = np.nan_to_num(s)
        return 1./s
    def gradient(self,k,t,X):
        output = np.zeros((len(self.w[0]),1))
        for n in range(len(X)):
            scale = self.softmax(n,k,X) - t[:,k][n] #Ynk - Tnk
            output += scale * self.phi(X[n])
        return output
    def hessian(self,k,X):
        output = np.zeros((len(self.w[0]),len(self.w[0])))
        for n in range(len(X)): 
            scale = self.softmax(n,k,X) * (1 - self.softmax(n,k,X))
            output += scale * (self.phi(X[n]).dot(self.phi(X[n]).T)) 
        return output
    def error(self,t,x):
        s = 0
        for n in range(x.shape[0]):
            for k in range(self.Num_class):
                s += t[n,k]*np.nan_to_num(np.log(self.softmax(n,k,x)))
        return -1*s
    def classify(self,x): 
        sfm = []
        for k in range(self.Num_class):
            tt = 0
            ak = self.w[k].T.dot(self.phi(x))
            for k in range(self.Num_class):
                aj = self.w[k,:].T.dot(self.phi(x))
                tt =tt+ np.nan_to_num(np.e**(aj - ak))
            sfm.append(1/tt)
        return sfm.index(max(sfm))
    def accuracy(self,x,t): 
        ac=0
        for i in range(x.shape[0]):
            if t[i,self.classify(x[i,:])] ==1:
                ac=ac+1
        return ac/x.shape[0]
    def Train(self, x, t, grad_fun = 'Newton'):
        self.w = np.zeros((self.Num_class,x.shape[1],1))
        cee = []
        acc = []
        epoch = 1
        while True:
            e = self.error(t, x)
            acc += [self.accuracy(x,t)]
            cee += [np.reshape(e,1)]
            epoch += 1
            if e < 0.001 or epoch>=self.Num_epoch:
                break
            for k in range(self.Num_class):
                print(self.hessian(k,x).shape)
                try:
                    self.w[k] = self.w[k] - np.linalg.inv(self.hessian(k,x)).dot(self.gradient(k,t,x))
                except:
                    self.w[k] = self.w[k] - np.linalg.pinv(self.hessian(k,x)).dot(self.gradient(k,t,x))

In [17]:
LM = LogisticModal(num_class)
LM.Train(Xtrain,ytrain)

[[463.51811878]]
1
(784, 784)
(784, 784)
(784, 784)
(784, 784)
(784, 784)
[[14.98745794]]
2
(784, 784)
(784, 784)
(784, 784)
(784, 784)
(784, 784)
[[1.98599358]]
3
(784, 784)
(784, 784)
(784, 784)
(784, 784)
(784, 784)
[[0.26804044]]
4
(784, 784)


KeyboardInterrupt: 

In [74]:
np.zeros((5,784,1))[0].shape

(784, 1)

In [46]:
phi(Xtrain[0,:])

array([[  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [ 36],
       [158],
       [111],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [147],
       [137],
       [ 29],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [120],
       [213],
       [185],
       [236],
       [197],
       [188],
       [219],
       [202],
       [174],
       [222],
       [ 71],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [220],
       [235],
       [233],
       [240],
       [238],
      